In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "train_25_mb_with_js_data/"
create_directory(data_directory)
create_directory("models/train_25_mb_with_js_data/")
create_directory("scalers_js")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 60
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 2700

INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/js_train_passive_trials.h5")
val_trials = read_dataset("data/js_val_passive_trials.h5")

100%|██████████| 1000/1000 [00:12<00:00, 83.06it/s]


In [6]:
train_trials.head()

AttributeError: 'list' object has no attribute 'head'

## Train model and plot loss and accuracy

In [6]:
N_MODELS = 25

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/train_25_mb_with_js_data/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/train_25_mb_with_js_data/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:01<00:00, 652.42it/s]
Train_loss: ([0.42116813 0.40227061]) Train_acc: ([82.68571429 75.82857143]) Val_acc: ([62.6 62. ]): 100%|██████████| 60/60 [12:09<00:00, 11.87s/it]
Train_loss: ([0.83519296 0.79798467]) Train_acc: ([59.68571429 56.77142857]) Val_acc: ([54.4 54.5]): 100%|██████████| 60/60 [11:46<00:00, 12.21s/it]
Train_loss: ([0.37885531 0.3208868 ]) Train_acc: ([83.4        87.02857143]) Val_acc: ([67.2 69.3]): 100%|██████████| 60/60 [11:57<00:00, 11.57s/it]
Train_loss: ([0.78588446 0.67491601]) Train_acc: ([64.14285714 57.85714286]) Val_acc: ([56.3 53.5]): 100%|██████████| 60/60 [11:31<00:00, 11.61s/it]
Train_loss: ([0.43748533 0.47207051]) Train_acc: ([74.45714286 78.05714286]) Val_acc: ([59.9 61.4]): 100%|██████████| 60/60 [11:44<00:00, 12.07s/it]
Train_loss: ([0.34404336 0.59407423]) Train_acc: ([84.57142857 51.2       ]) Val_acc: ([62.4 51.1]): 100%|██████████| 60/60 [11:41<00:00, 11.92s/it]
Train_loss: ([0.44283795 0.3393492 ]) Train_acc: ([81

## Save model and scaler

In [7]:
joblib.dump(scaler, "scalers_js/passive_dual_scaler.sk")

['scalers_js/passive_dual_scaler.sk']